In [1]:
import sys
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import RandomOverSampler

sys.path.append('/Users/nickocruz/Developer/EPL-market-inefficiency-forked/Nickos/src')
from Functions import *

DataPath = "../Data/Original/"

## Pre-Processing

- Taking orginal datasets:
  - Merging them on match_id with inner
    - inner: only keeps data that matches the match_id

In [3]:
df = construct_dataset(DataPath)
df = prepare_dataset(df)
print(df)

        match_id      date_x          home_team         away_team  result  \
0    EPL2425_001  2024-08-10            Arsenal         Newcastle       0   
1    EPL2425_002  2024-08-10        Aston Villa          Brighton       0   
2    EPL2425_003  2024-08-10        Bournemouth            Wolves       2   
3    EPL2425_004  2024-08-10            Burnley        Man United       2   
4    EPL2425_005  2024-08-10            Chelsea    Crystal Palace       0   
..           ...         ...                ...               ...     ...   
365  EPL2425_366  2025-05-19          Newcastle          Man City       0   
366  EPL2425_367  2025-05-19  Nottingham Forest  Sheffield United       1   
367  EPL2425_368  2025-05-19        Bournemouth            Wolves       2   
368  EPL2425_369  2025-05-19            Burnley          Brighton       0   
369  EPL2425_370  2025-05-19          Tottenham            Fulham       1   

         date_y  home_win_odds  draw_odds  away_win_odds          favorite 

,match_id,date,home_team,away_team,result,home_win_odds,draw_odds,away_win_odds,favorite,home_team_id,away_team_id,favorite_id,is_draw
0,EPL2425_001,2024-08-10,Arsenal,Newcastle,0,4.60,4.93,1.88,Newcastle,19,8,8,0
1,EPL2425_002,2024-08-10,Aston Villa,Brighton,0,3.09,5.36,2.26,Brighton,4,3,3,0
2,EPL2425_003,2024-08-10,Bournemouth,Wolves,2,2.19,5.90,3.06,Wolves,12,2,2,0
3,EPL2425_004,2024-08-10,Burnley,Man United,2,3.30,5.06,2.21,Man United,15,6,6,0
4,EPL2425_005,2024-08-10,Chelsea,Crystal Palace,0,4.18,3.36,2.41,Crystal Palace,17,9,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,EPL2425_366,2025-05-19,Newcastle,Man City,0,3.87,5.05,2.01,Man City,8,1,1,0
366,EPL2425_367,2025-05-19,Nottingham Forest,Sheffield United,1,2.12,4.75,3.70,Sheffield United,10,5,5,1
367,EPL2425_368,2025-05-19,Bournemouth,Wolves,2,2.27,5.86,2.93,Wolves,12,2,2,0
368,EPL2425_369,2025-05-19,Burnley,Brighton,0,2.91,4.72,2.52,Brighton,15,3,3,0


In [ ]:
class DrawPredictionNN(nn.Module):
    def __init__(self, input_size, hidden1=64, hidden2=32):
        super(DrawPredictionNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, 16)
        self.fc4 = nn.Linear(16,1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x1 = self.fc1(x)
        x2 = self.fc2(x1)
        x3 = self.fc3(x2)
        return self.relu(self.fc4(x3))